In [ ]:
#This note book is used for exploring dataset.
#这是用来对训练集数据进行探索的工作薄。

#linux commands to move dog/cat pictures into seperate folders
#使用linux命令将原始数据集移动到不同的文件夹
#mkdir cat
#find ./ -maxdepth 1 -name 'cat*.jpg' -exec mv {} ./cat \;
#mkdir dog
#find ./ -maxdepth 1 -name 'dog*.jpg' -exec mv {} ./dog \;

In [1]:
#Load training dataset
#加载训练图片数据集
from sklearn.datasets import load_files     
data = load_files('train')
import numpy as np
from keras.utils import np_utils
train_files = np.array(data['filenames'])
train_targets = np_utils.to_categorical(np.array(data['target']), 2)     

Using TensorFlow backend.


In [2]:
#extract dimension information from training dataset
#从训练图片训练集提取图片维度信息
import cv2
train_dimensions_list = []
for train_file in train_files:
    img = cv2.imread(train_file)
    dimensions = img.shape
    train_dimensions_list.append([train_file,dimensions[0],dimensions[1],dimensions[2]])

In [3]:
#check dimensions
#查看图片维度信息的统计数据
import pandas as pd
train_dimensions=pd.DataFrame(train_dimensions_list, columns = ["train_file", "width", "height", "depth"])
train_dimensions.describe()

,width,height,depth
count,25000.000000,25000.00000,25000.0
mean,360.478080,404.09904,3.0
std,97.019959,109.03793,0.0
min,32.000000,42.00000,3.0
25%,301.000000,323.00000,3.0
50%,374.000000,447.00000,3.0
75%,421.000000,499.00000,3.0
max,768.000000,1050.00000,3.0


In [42]:
#Use tableau public to perform further analysis on csv file
#将训练集图片维度信息存储为csv文件，在tableau中进一步可视化
train_dimensions.to_csv('train_dimension.csv')

In [15]:
#check label error using pre-trained resnet50
#this approach turned out to be not very efficient
#使用keras的resnet50预训练模型来对图片进行预测，进而对训练集标签错误进行复核。
#手工复核中发现这种方法带来的工作量太大，所以更改为使用更加先进的百度图片识别服务。
#这部分代码仅仅留存在这里作为参考，没有被实际使用。

#from keras.applications.resnet50 import ResNet50
#from keras.preprocessing import image
#from keras.applications.resnet50 import preprocess_input, decode_predictions
#import numpy as np

#model = ResNet50(weights='imagenet')
#train_labels = []

#for train_file,train_target in zip(train_files,train_targets):
#    img = image.load_img(train_file, target_size=(224, 224))
#    x = image.img_to_array(img)
#    x = np.expand_dims(x, axis=0)
#    x = preprocess_input(x)
#    preds = model.predict(x)
#    train_labels.append([train_file, decode_predictions(preds, top=1)[0][0][1]])
    
#import pandas as pd
#train_labels=pd.DataFrame(train_labels, columns = ["train_file", "predict"])
#train_labels.to_csv('train_label.csv')

In [24]:
#use baidu ai to predict labels for training dataset.
#使用百度图片识别服务来对训练集的图片进行标签预测

#01.initialization
#01.初始化
from aip import AipImageClassify

APP_ID = '' #value removed
API_KEY = '' #value removed
SECRET_KEY = '' #value removed

client = AipImageClassify(APP_ID, API_KEY, SECRET_KEY)

def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read()

train_labels_df = pd.DataFrame(pd.read_csv("train_label.csv"),columns = ['train_file','predict'])
train_labels = train_labels_df.values.tolist()
predicted_files = train_labels_df['train_file'].tolist()

In [111]:
#use baidu ai to predict labels for training dataset.
#使用百度图片识别服务来对训练集的图片进行标签预测

#02.execution
#02.执行标签预测

from ipywidgets import IntProgress
from IPython.display import display
import time

p = IntProgress()
errorCount=0
progressFull=train_files.size
display(p)

for train_file in train_files:
    if train_file not in predicted_files:
        image = get_file_content(train_file)
        options = {}
        options["top_num"] = 1
        try:
            classifyResult=client.animalDetect(image, options)
            train_labels.append([train_file,classifyResult['result'][0]['name']])            
        except:
            print(train_file)
            print(classifyResult)
            errorCount=errorCount+1
            if errorCount==30:
                break
        else:
            predicted_files.append(train_file)            
            p.value = float(len(train_labels))/float(progressFull)*100
            p.description = str(len(train_labels))+'/'+str(progressFull)

A Jupyter Widget

train/cat/cat.10392.jpg
{'log_id': 2214097746608673991, 'error_code': 216202, 'error_msg': 'image size error'}
train/dog/dog.10733.jpg
{'log_id': 3478175862374451154, 'error_code': 216202, 'error_msg': 'image size error'}
train/cat/cat.5534.jpg
{'log_id': 6767665921178003551, 'error_code': 216202, 'error_msg': 'image size error'}
train/dog/dog.4367.jpg
{'log_id': 9063937966059924784, 'error_code': 216202, 'error_msg': 'image size error'}
train/cat/cat.6614.jpg
{'log_id': 1846562680616028352, 'error_code': 216202, 'error_msg': 'image size error'}
train/dog/dog.1324.jpg
{'log_id': 6751093081619811531, 'error_code': 216202, 'error_msg': 'image size error'}
train/dog/dog.9246.jpg
{'log_id': 4213569367857841417, 'error_code': 216202, 'error_msg': 'image size error'}
train/dog/dog.11248.jpg
{'log_id': 8369342386938046913, 'error_code': 216202, 'error_msg': 'image size error'}
train/dog/dog.2652.jpg
{'log_id': 1822557284218531936, 'error_code': 216202, 'error_msg': 'image size error'}
train/c

In [112]:
#use baidu ai to predict labels for training dataset.
#使用百度图片识别服务来对训练集的图片进行标签预测

#03.store results for further analysis
#02.将结果存储起来，然后做进一步分析
import pandas as pd
train_labels_df=pd.DataFrame(train_labels, columns = ["train_file", "predict"])
train_labels_df.to_csv('train_label.csv')